In [1]:
import os
import pickle
import re
import pprint
import numpy as np
import sympy as sp
import openturns as ot
# Display the final graph
import openturns.viewer as viewer
import matplotlib.pyplot as plt
import trimesh as tr
from importlib import reload
from functools import partial

from math import pi
from joblib import Parallel, delayed
from importlib import reload
from IPython.display import display, clear_output
from time import time
from sympy.printing import latex
from trimesh import viewer as trview
import sklearn

from scipy.optimize import OptimizeResult, minimize, basinhopping, differential_evolution, brute, shgo, check_grad, approx_fprime

import tqdm
import otaf

from gldpy import GLD

ot.Log.Show(ot.Log.NONE)
np.set_printoptions(suppress=True)
ar = np.array

# Notebook for the analysis of a system comprised of N + 2 parts, 2 plates with N = N1 x N2 holes, and N pins. 

### Defintion on global descriptive parameters

In [2]:
NX = 2 ## Number of holes on x axis
NY = 2 ## Number of holes on y axis
Dext = 20 ## Diameter of holes in mm
Dint = 19.8 ## Diameter of pins in mm
EH = 50 ## Distance between the hole axises
LB = 25 # Distance between border holes axis and edge.
hPlate = 30 #Height of the plates in mm
hPin = 60 #Height of the pins in mm

CIRCLE_RESOLUTION = 16 # NUmber of points to model the contour of the outer holes

### Defining and constructing the system data dictionary

The plates have NX * NY + 1 surfaces. The lower left point has coordinate 0,0,0

We only model the surfaces that are touching. 

In [3]:
N_PARTS = NX * NY * 2
LX = (NX - 1) * EH + 2*LB
LY = (NY - 1) * EH + 2*LB

contour_points = ar([[0,0,0],[LX,0,0],[LX,LY,0],[0,LY,0]])

R0 = ar([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
x_, y_, z_ = R0[0], R0[1], R0[2]

Frame1 = ar([z_,y_,-x_])
Frame2 = ar([-z_,y_,x_])

First we define the base part dictionaries for the upper and lower plate, without holes

In [4]:
system_data = {
    "PARTS" : {
        '0' : {
            "a" : {
                "FRAME": Frame1,
                "POINTS": {'A0' : ar([0,0,0]),
                           'A1' : ar([LX,0,0]),
                           'A2' : ar([LX,LY,0]),
                           'A3' : ar([0,LY,0]),
                        },
                "TYPE": "plane",
                "INTERACTIONS": ['P1a'],
                "CONSTRAINTS_D": ["PERFECT"],
                "CONSTRAINTS_G": ["SLIDING"],            
            }
        },
        '1' : {
            "a" : {
                "FRAME": Frame2,
                "POINTS": {'A0' : ar([0,0,0]),
                           'A1' : ar([LX,0,0]),
                           'A2' : ar([LX,LY,0]),
                           'A3' : ar([0,LY,0]),
                        },
                "TYPE": "plane",
                "INTERACTIONS": ['P0a'],
                "CONSTRAINTS_D": ["PERFECT"],
                "CONSTRAINTS_G": ["SLIDING"],            
            }
        }  
    },
    "LOOPS": {
        "COMPATIBILITY": {
        },
    },
    "GLOBAL_CONSTRAINTS": "3D",
}

Then we iterate over the pin dimensions NX and NY, and create the corresponding holes and pins. At the same time there is 1 loop per pin

In [5]:
alpha_gen = otaf.common.alphabet_generator()
next(alpha_gen) # skipping 'a' as it has already been used above
part_id = 2 # Start part index for pins
for i in range(NX):
    for j in range(NY):
        pcor = ar([LB+i*EH, LB+j*EH, 0]) # Point coordinate for hole / pins
        slab = next(alpha_gen) # Surface label, same for each mating pin so its easeir to track
        # Creating pin
        system_data["PARTS"][str(part_id)] = {}
        system_data["PARTS"][str(part_id)][slab] = {
            "FRAME": Frame1, # Frame doesn't really matter, as long as x is aligned on the axis
            "ORIGIN": pcor, 
            "TYPE": "cylinder",
            "RADIUS": Dint / 2,
            "EXTENT_LOCAL": {"x_max": hPin/2, "x_min": -hPin/2},
            "INTERACTIONS": [f"P0{slab}", f"P1{slab}"], 
            "SURFACE_DIRECTION": "centrifugal",
            "CONSTRAINTS_D": ["PERFECT"], # No defects on the pins
            "BLOCK_ROTATIONS_G": 'x', # The pins do not rotate around their axis
            "BLOCK_TRANSLATIONS_G": 'x', # The pins do not slide along their axis
        }
        # Adding hole to part 0
        system_data["PARTS"]["0"][slab] = {
            "FRAME": Frame1,
            "ORIGIN": pcor, 
            "TYPE": "cylinder",
            "RADIUS": Dext / 2,
            "EXTENT_LOCAL": {"x_max": hPin/2, "x_min": -hPin/2},
            "INTERACTIONS": [f"P{part_id}{slab}"], 
            "SURFACE_DIRECTION": "centripetal",
        }
        # Adding hole to part 1
        system_data["PARTS"]["1"][slab] = {
            "FRAME": Frame2,
            "ORIGIN": pcor, 
            "TYPE": "cylinder",
            "RADIUS": Dext / 2,
            "EXTENT_LOCAL": {"x_max": hPin/2, "x_min": -hPin/2},
            "INTERACTIONS": [f"P{part_id}{slab}"],
            "SURFACE_DIRECTION": "centripetal",
        }
        # Construct Compatibility loop
        loop_id = f"L{part_id-1}"
        formater = lambda i,l : f"P{i}{l}{l.upper()}0" 
        system_data["LOOPS"]["COMPATIBILITY"][loop_id] = f"P0aA0 -> {formater(0,slab)} -> {formater(part_id,slab)} -> {formater(1,slab)} -> P1aA0"
        part_id += 1  

In [6]:
SDA = otaf.AssemblyDataProcessor(system_data)
SDA.generate_expanded_loops()

In [7]:
CLH = otaf.CompatibilityLoopHandling(SDA)
compatibility_expressions = CLH.get_compatibility_expression_from_FO_matrices()

In [8]:
ILH = otaf.InterfaceLoopHandling(SDA, CLH, circle_resolution=CIRCLE_RESOLUTION)
interface_constraints = ILH.get_interface_loop_expressions()

Processing part 0, surface b for cylinder-to-cylinder interactions.
usedGMatDat [['0', 'b', 'B0', '2', 'b', 'B0']]
Found 1 used gap matrices.
unusedGMatDat [['0', 'b', 'B1', '2', 'b', 'B1'], ['0', 'b', 'B2', '2', 'b', 'B2']]
Found 2 unused gap matrices.
Matching used and unused gap matrices: GP0bB0P2bB0 with GP0bB1P2bB1
Matching used and unused gap matrices: GP0bB0P2bB0 with GP0bB2P2bB2
Generated 32 interaction equations for current matching.
Total interaction equations generated: 32
Processing part 0, surface c for cylinder-to-cylinder interactions.
usedGMatDat [['0', 'c', 'C0', '3', 'c', 'C0']]
Found 1 used gap matrices.
unusedGMatDat [['0', 'c', 'C1', '3', 'c', 'C1'], ['0', 'c', 'C2', '3', 'c', 'C2']]
Found 2 unused gap matrices.
Matching used and unused gap matrices: GP0cC0P3cC0 with GP0cC1P3cC1
Matching used and unused gap matrices: GP0cC0P3cC0 with GP0cC2P3cC2
Generated 32 interaction equations for current matching.
Total interaction equations generated: 32
Processing part 0, sur

In [9]:
SOCAM = otaf.SystemOfConstraintsAssemblyModel(
    compatibility_expressions, interface_constraints
)

SOCAM.embedOptimizationVariable()

print(len(SOCAM.deviation_symbols), SOCAM.deviation_symbols)

32 [v_d_0, w_d_0, beta_d_0, gamma_d_0, v_d_2, w_d_2, beta_d_2, gamma_d_2, v_d_5, w_d_5, beta_d_5, gamma_d_5, v_d_7, w_d_7, beta_d_7, gamma_d_7, v_d_8, w_d_8, beta_d_8, gamma_d_8, v_d_10, w_d_10, beta_d_10, gamma_d_10, v_d_11, w_d_11, beta_d_11, gamma_d_11, v_d_13, w_d_13, beta_d_13, gamma_d_13]


## Construction of the stochastic model of the defects. (old lambda approach)

In [10]:
tol = 0.1 * np.sqrt(2)
Cm = 1  # Process capability

# Defining the uncertainties on the position and orientation uncertainties.
sigma_e_pos = tol / (6 * Cm)
theta_max = tol / hPlate
sigma_e_theta = (2 * theta_max) / (6 * Cm)

In [11]:
RandDeviationVect = otaf.distribution.get_composed_normal_defect_distribution(
    defect_names=SOCAM.deviation_symbols,
    sigma_dict = {"alpha":sigma_e_theta, 
                  "beta":sigma_e_theta,
                  "gamma":sigma_e_theta, 
                  "u":sigma_e_pos, 
                  "v":sigma_e_pos, 
                  "w":sigma_e_pos})
dim_devs = int(RandDeviationVect.getDimension())

# Instead of using a surrogate and using big samples, now we'll fit a GLD on a small sample of the slack to estimate the probability of failure.

## Now lets first to a double loop monte-carlo to explore the full space of the parameters (using the intermediate lambda space) and the stochastic space, to be able to draw the full P-Box of the slack

### The below sampling approach we used previously is absoluetly false for a cylindrical feature. The constraint function on the 4 degrees of freedom is λu2​+λv2​+λα2​+λβ2​+2(λu​λα​+λv​λβ​)−1=0.

In [12]:
# Big sample in the space of the lambdas (parameters).
D_lambd = len(SOCAM.deviation_symbols)
lambda_vect_unconditioned = ot.ComposedDistribution([ot.Uniform(0, 1)] * D_lambd)
lambda_vect_unconditioned.setDescription(list(map(str, SOCAM.deviation_symbols)))
N_lambda = 20
lambda_sample_unconditioned = otaf.sampling.generateLHSExperiment(lambda_vect_unconditioned ,N_lambda, 999)
#lambda_sample_unconditioned = lambda_sample_unconditioned_generator.generate()
lambda_sample_random = lambda_vect_unconditioned.getSample(N_lambda)
lambda_sample_conditioned = otaf.sampling.condition_lambda_sample(lambda_sample_random, squared_sum=True)

In [13]:
e(èi)

NameError: name 'e' is not defined

## Dual loop monte carlo with GLD 

In [ ]:
bounds = None
SEED_MC_PF = 6436431
SIZE_MC_PF = 20000 #int(1e6) #1e4
s_values = np.zeros((N_lambda, SIZE_MC_PF)) # For each MC point we have a s value
GLD_parameters = [] # We need the parameters of the generalized lambda distribution.
failure_probabilities = []
# Generalized lambda distribution object for fitting
gld = GLD('VSL')

start_time = time()  # Record the start time
for i in range(N_lambda):
    print(f"Doing iteration {i} of {N_lambda}")
    if i>0:
        fp = failure_probabilities
        print(f"Failure probability i-1 : {fp[i-1]}, Min: {min(fp)}, / Max: {max(fp)}")
        print("s_mean: ", s_values.mean().round(3), "s_min: ", np.nanmin(s_values).round(3), "s_max: ", np.nanmax(s_values).round(3))

    ot.RandomGenerator.SetSeed(SEED_MC_PF)
    deviation_samples = np.array(RandDeviationVect.getSample(SIZE_MC_PF)) * np.array(
        lambda_sample_conditioned[i]
    )
    optimizations = otaf.uncertainty.compute_gap_optimizations_on_sample(
            SOCAM,
            deviation_samples,
            bounds=bounds,
            n_cpu=-1,
            progress_bar=True,
        )
    
    slack = np.array([opt.fun for opt in optimizations], dtype=float)*-1 #Normally there aren"t any nans.
    s_values[i,:] = slack
    GLD_parameters.append(gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False))
    failure_probabilities.append(gld.CDF_num(0, GLD_parameters[i]))


print(f"Done {len (lambda_sample_conditioned)} experiments.")
print(f"Elapsed time: {time() - start_time:.3f} seconds.")

In [ ]:
X = otaf.sampling.find_best_worst_quantile(np.array(lambda_sample_conditioned), np.array(failure_probabilities), 0.1)
(best_5p_lambda, best_5p_res), (worst_5p_lambda, worst_5p_res) = X

In [ ]:
distributions = [ot.UserDefined(ot.Sample(s[:,np.newaxis])) for s in s_values]
x_min, x_max = -0.03, 0.11
sup_data, inf_data = otaf.distribution.compute_sup_inf_distributions(distributions, x_min, x_max)

In [ ]:
# Initialize colors and legends for each lambda sample
colors = [["grey"]] * lambda_sample_conditioned.getSize()
legends = [""] * lambda_sample_conditioned.getSize()

# Plot the combined CDF with additional curves for the envelopes
graph_full = otaf.plotting.plot_combined_CDF(distributions, x_min, x_max, colors, legends)
graph_full = otaf.plotting.set_graph_legends(
    graph_full,
    x_title="j",
    y_title="P",
    title="Slack P-BOX 32 dimensions",
    legends=legends
)

# Add the upper and lower envelopes to the graph
graph_full.add(ot.Curve(inf_data, "blue", "solid", 1.5, "lower envelope"))
graph_full.add(ot.Curve(sup_data, "red", "solid", 1.5, "upper envelope"))
view = viewer.View(graph_full, pixelsize=(1100, 750))

In [13]:
min(failure_probabilities)

NameError: name 'failure_probabilities' is not defined

# Now we'll use the other lambda modelization, a bit more difficutl to sample

In [14]:
# All the extremal values for lambda, in 32 dimensions :


In [15]:
lambda_limit_sample = otaf.sampling.generate_imprecise_probabilistic_samples([4]*8, -1, discretization=1)
lambda_limit_sample_sub_choice = [next(lambda_limit_sample) for _ in range(60000)]
lambda_limit_sample_sub_choice = np.stack(lambda_limit_sample_sub_choice)
print(lambda_limit_sample_sub_choice.shape)

(60000, 32)


In [16]:
from scipy.optimize import fsolve
def solve_for_lambda_beta(lambda_u, lambda_v, lambda_alpha):
    """
    Solve the equation for lambda_beta given lambda_u, lambda_v, and lambda_alpha.
    """
    def equation(lambda_beta):
        return (lambda_u**2 + lambda_v**2 + lambda_alpha**2 + lambda_beta**2 +
                2 * (lambda_u * lambda_alpha + lambda_v * lambda_beta) - 1)
    
    # Use a root-finding method to solve for lambda_beta
    lambda_beta_initial_guess = 0.5
    lambda_beta_solution = fsolve(equation, lambda_beta_initial_guess)[0]
    
    return lambda_beta_solution

def generate_points_on_surface(num_points=1000):
    """
    Generate points on the surface by solving for lambda_beta
    given random values for lambda_u, lambda_v, and lambda_alpha.
    
    Returns:
    - A numpy array of shape (num_valid_points, 4) where each row is 
      (lambda_u, lambda_v, lambda_alpha, lambda_beta).
    """
    # Generate random values for lambda_u, lambda_v, lambda_alpha in [0, 1]
    lambda_u = np.random.rand(num_points)
    lambda_v = np.random.rand(num_points)
    lambda_alpha = np.random.rand(num_points)
    
    # Assume lambda_beta = 0 and check if the inequality is satisfied
    inequality_values = lambda_u**2 + lambda_v**2 + lambda_alpha**2 + 2 * (lambda_u * lambda_alpha)

    # Filter valid points where inequality holds
    valid_mask = inequality_values <= 1
    lambda_u_valid = lambda_u[valid_mask]
    lambda_v_valid = lambda_v[valid_mask]
    lambda_alpha_valid = lambda_alpha[valid_mask]
    
    # Solve for lambda_beta for valid points
    lambda_beta_valid = np.array([solve_for_lambda_beta(u, v, a) 
                                  for u, v, a in zip(lambda_u_valid, lambda_v_valid, lambda_alpha_valid)])

    # Only keep points where the solved lambda_beta is between 0 and 1
    final_mask = (lambda_beta_valid >= 0) & (lambda_beta_valid <= 1)
    final_points = np.vstack((lambda_u_valid[final_mask],
                              lambda_v_valid[final_mask],
                              lambda_alpha_valid[final_mask],
                              lambda_beta_valid[final_mask])).T
    
    return final_points

# Generate the points on the surface
points = generate_points_on_surface(num_points=20000)

# Call the pair_plot function (assuming it's already defined)
labels = ['lambda_u', 'lambda_v', 'lambda_alpha', 'lambda_beta']
#otaf.plotting.pair_plot(points, labels)

In [17]:
random_lambda_sample = [points[np.random.choice(points.shape[0], 200, replace=False)] for _ in range(8)]

In [18]:
random_lambda_sample = np.hstack(random_lambda_sample)

In [19]:
random_lambda_sample.shape

(200, 32)

In [20]:
N_lambda = random_lambda_sample.shape[0]
bounds = None
SEED_MC_PF = 6436431
SIZE_MC_PF = 10000 #int(1e6) #1e4
s_values = np.zeros((N_lambda, SIZE_MC_PF)) # For each MC point we have a s value
GLD_parameters = [] # We need the parameters of the generalized lambda distribution.
failure_probabilities = []
# Generalized lambda distribution object for fitting
gld = GLD('VSL')

start_time = time()  # Record the start time
for i in range(N_lambda):
    print(f"Doing iteration {i} of {N_lambda}")
    if i>0:
        fp = failure_probabilities
        print(f"Failure probability i-1 : {fp[i-1]}, Min: {min(fp)}, / Max: {max(fp)}")
        print("s_mean: ", s_values.mean().round(3), "s_min: ", np.nanmin(s_values).round(3), "s_max: ", np.nanmax(s_values).round(3))

    ot.RandomGenerator.SetSeed(SEED_MC_PF)
    deviation_samples = np.array(RandDeviationVect.getSample(SIZE_MC_PF)) * random_lambda_sample[i]
    optimizations = otaf.uncertainty.compute_gap_optimizations_on_sample(
            SOCAM,
            deviation_samples,
            bounds=bounds,
            n_cpu=-1,
            progress_bar=True,
        )
    
    slack = np.array([opt.fun for opt in optimizations], dtype=float)*-1 #Normally there aren"t any nans.
    s_values[i,:] = slack
    GLD_parameters.append(gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False))
    failure_probabilities.append(gld.CDF_num(0, GLD_parameters[i]))


print(f"Done {len (N_lambda)} experiments.")
print(f"Elapsed time: {time() - start_time:.3f} seconds.")

Doing iteration 0 of 200


  0%|          | 0/10000 [00:00<?, ?it/s]

Doing iteration 1 of 200
Failure probability i-1 : [0.00000596], Min: [0.00000596], / Max: [0.00000596]
s_mean:  0.0 s_min:  0.0 s_max:  0.091


  0%|          | 0/10000 [00:00<?, ?it/s]

Doing iteration 2 of 200
Failure probability i-1 : [0.00000596], Min: [0.00000596], / Max: [0.00000596]
s_mean:  0.001 s_min:  0.0 s_max:  0.092


...

/home/kramer84/anaconda3/envs/anr-dev/lib/python3.11/site-packages/gldpy/_gld_module.py:166: RuntimeWarning: divide by zero encountered in power
  return param[0] + ((1 - param[2])*(y**param[3] - 1)/param[3] - param[2]*((1-y)**param[3] - 1)/param[3])*param[1]
/home/kramer84/anaconda3/envs/anr-dev/lib/python3.11/site-packages/gldpy/_gld_module.py:166: RuntimeWarning: divide by zero encountered in scalar power
  return param[0] + ((1 - param[2])*(y**param[3] - 1)/param[3] - param[2]*((1-y)**param[3] - 1)/param[3])*param[1]


  0%|          | 0/10000 [00:00<?, ?it/s]

Doing iteration 199 of 200
Failure probability i-1 : [0.00000596], Min: [0.], / Max: [0.00104828]
s_mean:  0.07 s_min:  -0.02 s_max:  0.095


  0%|          | 0/10000 [00:00<?, ?it/s]

TypeError: object of type 'int' has no len()

In [21]:
X = otaf.sampling.find_best_worst_quantile(np.array(lambda_sample_conditioned), np.array(failure_probabilities), 0.1)
(best_5p_lambda, best_5p_res), (worst_5p_lambda, worst_5p_res) = X

In [22]:
distributions = [ot.UserDefined(ot.Sample(s[:,np.newaxis])) for s in s_values]
x_min, x_max = -0.03, 0.11
sup_data, inf_data = otaf.distribution.compute_sup_inf_distributions(distributions, x_min, x_max)

In [23]:
%matplotlib qt

In [27]:
# Initialize colors and legends for each lambda sample
colors = [["grey"]] * N_lambda
legends = [""] * N_lambda

# Plot the combined CDF with additional curves for the envelopes
graph_full = otaf.plotting.plot_combined_CDF(distributions, x_min, x_max, colors, legends)
graph_full = otaf.plotting.set_graph_legends(
    graph_full,
    x_title="slack",
    y_title="$P_{slack}$",
    title="Slack P-BOX 32 dimensions",
    legends=legends
)

# Add the upper and lower envelopes to the graph
graph_full.add(ot.Curve(inf_data, "blue", "solid", 1.5, "lower envelope"))
graph_full.add(ot.Curve(sup_data, "red", "solid", 1.5, "upper envelope"))
view = viewer.View(graph_full, pixelsize=(1100, 750))

In [28]:
# %matplotlib qt
fig = plt.figure(dpi=150)

ax = fig.add_subplot(1, 1, 1)
ax.plot(sup_data[:, 0], sup_data[:, 1], color="tab:orange", label="upper envelope")
ax.plot(inf_data[:, 0], inf_data[:, 1], color="tab:blue", label="lower envelope")
ax.grid(True)
ax.fill_between(
    inf_data[:, 0], inf_data[:, 1], sup_data[:, 1], color="gray", alpha=0.3
)
ax.set_xlabel("slack")
ax.set_ylabel("$P_{slack}$")
ax.legend()
ax.set_title("Slack P-Box 3D Assembly 32D")

Text(0.5, 1.0, 'Slack P-Box 3D Assembly 32D')

In [16]:
print(f"Minimum failure probability: {np.min(failure_probabilities):.5e}, Maximum failure probability: {np.max(failure_probabilities):.5e}")

NameError: name 'failure_probabilities' is not defined

# Global optimization basinhopping with GLD

In [12]:
SIZE_MC_PF = 10000 #int(1e6) #1e4
sample_gld = np.array(RandDeviationVect.getSample(SIZE_MC_PF))
scale_factor = 1.0
GLD_parameters = [] # We need the parameters of the generalized lambda distribution.

# Generalized lambda distribution object for fitting
gld = GLD('VSL')

def model_base(x, sample=sample_gld):
    # Model without surrogate, to get slack
    x = sample * np.sqrt(x[np.newaxis, :])
    optimization_variables = otaf.uncertainty.compute_gap_optimizations_on_sample_batch(
        constraint_matrix_generator=SOCAM,
        deviation_array=x,
        batch_size=500,
        n_cpu=-1,
        progress_bar=True,
        verbose=0,
        dtype="float32",
    )
    slack_values = optimization_variables[:,-1]
    return slack_values

@otaf.optimization.scaling(scale_factor)
def optimization_function_mini(x, model=model_base):
    # Here we search the minimal probability of failure
    slack = model(x)
    gld_params = gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False)
    if np.any(np.isnan(gld_params)):
        failure_probability = np.where(slack<0,1,0).mean()
    else :
        print("gld_params:", gld_params)
        failure_probability = gld.CDF_num(0, gld_params)
    return failure_probability

@otaf.optimization.scaling(scale_factor)
def optimization_function_maxi(x, model=model_base):
    # Here we search the maximal probability of failure so negative output
    slack = model(x)
    gld_params = gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False)
    if np.any(np.isnan(gld_params)):
        failure_probability = np.where(slack<0,1,0).mean()
    else :
        print("gld_params:", gld_params)
        failure_probability = gld.CDF_num(0, gld_params)
    return failure_probability*-1

In [13]:
import numpy as np
import scipy.optimize

def cylinder_feature_constraint(la):
    """Evaluates the cylinder constraint for a given lambda array."""
    sum_squares = np.dot(la, la)  # Equivalent to la[0]**2 + la[1]**2 + la[2]**2 + la[3]**2
    cross_terms = 2 * (la[0] * la[2] + la[1] * la[3])
    return sum_squares + cross_terms - 1

def cylinder_feature_constraint_jacobian(la):
    """Computes the analytical Jacobian of the cylinder feature constraint."""
    u, v, alpha, beta = la
    return np.array([
        2 * u + 2 * alpha,  # df/du
        2 * v + 2 * beta,   # df/dv
        2 * alpha + 2 * u,  # df/dalpha
        2 * beta + 2 * v    # df/dbeta
    ])

def assembly_feature_constraint(la8):
    """Applies the cylinder constraint to each 4-element slice of the lambda array, returning a vector of constraints."""
    return np.array([cylinder_feature_constraint(la8[i : i + 4]) for i in range(0, len(la8), 4)])

def assembly_feature_constraint_jacobian(la8):
    """Constructs the Jacobian matrix for the vector of constraints, with each row representing the Jacobian of one cylinder constraint."""
    # Initialize a list to hold each constraint's Jacobian as a row in the matrix
    jacobian = []
    # Compute Jacobian for each 4-element segment in the lambda array
    for i in range(0, len(la8), 4):
        row_jacobian = np.zeros(len(la8))  # Create a zeroed row of the appropriate length
        row_jacobian[i:i+4] = cylinder_feature_constraint_jacobian(la8[i:i+4])
        jacobian.append(row_jacobian)
    return np.array(jacobian)


def cylinder_feature_constraint_hessian(la):
    """Computes the Hessian of the cylinder constraint for a given 4-element lambda array."""
    # Extract variables for readability
    u, v, alpha, beta = la
    # The Hessian matrix is symmetric, so we only need to define the unique elements
    hessian = np.array([
        [2, 0, 2, 0],  # d^2f/du^2, d^2f/du dv, d^2f/du dalpha, d^2f/du dbeta
        [0, 2, 0, 2],  # d^2f/dv du, d^2f/dv^2, d^2f/dv dalpha, d^2f/dv dbeta
        [2, 0, 2, 0],  # d^2f/dalpha du, d^2f/dalpha dv, d^2f/dalpha^2, d^2f/dalpha dbeta
        [0, 2, 0, 2]   # d^2f/dbeta du, d^2f/dbeta dv, d^2f/dbeta dalpha, d^2f/dbeta^2
    ])
    return hessian

def assembly_feature_constraint_hessian(la8):
    """Constructs the full Hessian matrix for the vector of constraints in the assembly."""
    n_constraints = len(la8) // 4
    hessian_blocks = []
    
    # Build the Hessian by iterating over each 4-element segment
    for i in range(n_constraints):
        block_hessian = cylinder_feature_constraint_hessian(la8[i*4:i*4+4])
        # Place the block into the full Hessian matrix for the constraint function
        block_hessian_expanded = np.zeros((len(la8), len(la8)))
        block_hessian_expanded[i*4:i*4+4, i*4:i*4+4] = block_hessian
        hessian_blocks.append(block_hessian_expanded)
    
    # Sum the block Hessians to form the full Hessian matrix
    full_hessian = np.sum(hessian_blocks, axis=0)
    return full_hessian

# Define the nonlinear constraint with the updated vector-valued function and Jacobian
nonLinearConstraint = scipy.optimize.NonlinearConstraint(
    fun=assembly_feature_constraint,
    lb=0.0,
    ub=0.0,
    jac=assembly_feature_constraint_jacobian,
    hess=assembly_feature_constraint_hessian
)

In [14]:
constraint_checker = otaf.optimization.create_constraint_checker(nonLinearConstraint,0)
opt_storage = otaf.optimization.OptimizationStorage(scipy.optimize.Bounds(lb=0.0,ub=1.0), constraint_checker)
step_taking = otaf.optimization.StepTaking(opt_storage, SOCAM.deviation_symbols)
accept_test = otaf.optimization.AcceptTest(opt_storage, SOCAM.deviation_symbols)
callback = otaf.optimization.Callback(opt_storage)

In [15]:
# Basinhopping for the maximization function using COBYQA
x0_maxi = [0.25] * RandDeviationVect.getDimension()  # Initial guess

# Update minimizer_kwargs_maxi to use COBYQA
minimizer_kwargs_maxi = {
    "method": "COBYQA",   # Use COBYQA method
    "jac": False,         # COBYQA doesn't use Jacobians
    "args": (model_base,),     # Update args to match COBYQA requirements
    "constraints": nonLinearConstraint,
    "bounds": scipy.optimize.Bounds(lb=0.0,ub=1.0),
    "options": {
        "f_target": -0.2, 
        "maxiter": 1000,
        "maxfev": 4000,
        "feasibility_tol": 1e-3,
        "initial_tr_radius": 1,
        "final_tr_radius": 1e-2,
        "disp": True
    }
}

# Running basinhopping with COBYQA as the local optimizer
res_maxi = basinhopping(
    optimization_function_maxi, x0_maxi,
    niter=5,
    T=1,
    stepsize=3.0,
    niter_success=19,
    interval=5,
    minimizer_kwargs=minimizer_kwargs_maxi,
    disp=True,
    #take_step=step_taking,
    #accept_test=accept_test,
    #callback=callback
)

print("Maximization Result with COBYQA:")
print(res_maxi)

Starting the optimization procedure.
Initial trust-region radius: 1.0.
Final trust-region radius: 0.01.
Maximum number of function evaluations: 4000.
Maximum number of iterations: 1000.



  0%|          | 0/10000 [00:00<?, ?it/s]

wrapper([ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]) = -0.0
assembly_feature_constraint([ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]) = [-1.000e+00 -1.000e+00 ... -1.000e+00 -1.000e+00]


/home/ksimady/anaconda3/envs/anr-dev/lib/python3.11/site-packages/gldpy/_gld_module.py:1026: RuntimeWarning: invalid value encountered in scalar divide
  return l1,l2,l3/l2,l4/l2
/home/ksimady/anaconda3/envs/anr-dev/lib/python3.11/site-packages/gldpy/_gld_module.py:1026: RuntimeWarning: divide by zero encountered in scalar divide
  return l1,l2,l3/l2,l4/l2
/home/ksimady/anaconda3/envs/anr-dev/lib/python3.11/site-packages/gldpy/_gld_module.py:1095: RuntimeWarning: invalid value encountered in scalar add
  if a4**2+98*a4 +1 <0:
/home/ksimady/anaconda3/envs/anr-dev/lib/python3.11/site-packages/gldpy/_gld_module.py:1097: RuntimeWarning: invalid value encountered in scalar add
  p4 = np.array([(3+7*a4 + np.sqrt(a4**2+98*a4 +1))/(2*(1-a4)), (3+7*a4 - np.sqrt(a4**2+98*a4 +1))/(2*(1-a4))])


  0%|          | 0/10000 [00:00<?, ?it/s]

...

gld_params: [0.05827131 0.02428555 0.29588105 0.14145451]
wrapper([ 8.558e-22  4.279e-19 ...  2.151e-01  9.766e-01]) = -0.009482663065780094
assembly_feature_constraint([ 8.558e-22  4.279e-19 ...  2.151e-01  9.766e-01]) = [-3.658e-06  0.000e+00 ...  5.011e-05 -1.223e-06]


  0%|          | 0/10000 [00:00<?, ?it/s]

gld_params: [0.05828538 0.02430859 0.29564355 0.14189038]
wrapper([ 0.000e+00  0.000e+00 ...  2.160e-01  9.764e-01]) = -0.009491352073161317
assembly_feature_constraint([ 0.000e+00  0.000e+00 ...  2.160e-01  9.764e-01]) = [ 8.569e-05  1.050e-05 ...  1.491e-03  6.617e-04]


  0%|          | 0/10000 [00:00<?, ?it/s]

gld_params: [0.05826325 0.02427965 0.29596492 0.14134085]
wrapper([ 2.768e-03  1.827e-19 ...  2.126e-01  9.771e-01]) = -0.009483384986557902
assembly_feature_constraint([ 2.768e-03  1.827e-19 ...  2.126e-01  9.771e-01]) = [-7.732e-06  0.000e+00 ...  5.863e-05  2.641e-06]

The lower bound for the trust-region radius has been reached.
Number of function evaluations: 305.
Number of iterations: 138.
Least value of wrapper: -0.009483384986557902.
Maximum constraint violation: 5.863233097658238e-05.
Corresponding point: [ 2.768e-03  1.827e-19 ...  2.126e-01  9.771e-01].
basinhopping step 5: f -0.00948338 trial_f -0.00948338 accepted 1  lowest_f -0.00948338
found new global minimum on step 5 with function value -0.00948338
Maximization Result with COBYQA:
                    message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: -0.009483384986557902
                          x: [ 2.768e-03  1.827e-19 ... 

In [16]:
res_maxi.x

array([0.00276788, 0.        , 0.        , 0.9999923 , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.00156387,
       0.99999915, 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.9644161 , 0.26441345,
       0.        , 0.        , 0.9562096 , 0.29270189, 0.        ,
       0.        , 0.22788806, 0.97371745, 0.        , 0.        ,
       0.21263898, 0.97713218])

In [19]:
optimization_function_maxi(np.array([0,0,1,0]*8))

  0%|          | 0/10000 [00:00<?, ?it/s]

gld_params: [0.05921167 0.02677207 0.32207694 0.16452816]


array([-0.00921768])